In [19]:
# 기존 FashionMNIST -> MNIST 변경
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# 방식 동일 / MNIST 변경
# 데이터 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))
])

In [4]:
# MNIST 숫자 데이터셋 로딩
train_data = datasets.MNIST(root='./data', train = True, download = True, transform = transform)
test_data = datasets.MNIST(root='./data', train = False, download = True, transform = transform)

# Batch Size 증가 : 64 -> 128 (성능 향상 목적)
train_loader = DataLoader(train_data, batch_size = 128, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 128, shuffle = False)

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.12MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 135kB/s]
100%|██████████| 1.65M/1.65M [00:06<00:00, 246kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.03MB/s]


In [9]:
# MLP 모델 정의
class MLP(nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    self.model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(28*28, 512), # 히든 노드 수 증가
        nn.ReLU(),
        nn.Dropout(0.3),       # 과적합 방지를 위한 드롭아웃
        nn.Linear(512, 256),   # 추가 히든 레이어
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, 10)     # 출력 : 숫자 0~9 (10개 클래스)
    )
  def forward(self, x):
    return self.model(x)

model = MLP().to(device)

In [10]:
# 최적화 SGD -> Adam, 학습률 조절
# 손실 함수 및 최적화 함수
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
# 학습 루프
def train(model, loader):
  model.train()
  total_loss = 0
  for x, y in loader:
    x, y = x.to(device), y.to(device)

    optimizer.zero_grad()
    output = model(x)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  return total_loss / len(loader)

In [16]:
# 평가 함수
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            output = model(x)
            pred = output.argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    return correct / total

In [17]:
# 학습 실행
for epoch in range(10):
    loss = train(model, train_loader)
    acc = evaluate(model, test_loader)
    print(f"Epoch {epoch+1} | Loss: {loss:.4f} | Test Accuracy: {acc:.4f}")

Epoch 1 | Loss: 0.4035 | Test Accuracy: 0.9445
Epoch 2 | Loss: 0.2004 | Test Accuracy: 0.9626
Epoch 3 | Loss: 0.1622 | Test Accuracy: 0.9674
Epoch 4 | Loss: 0.1427 | Test Accuracy: 0.9671
Epoch 5 | Loss: 0.1291 | Test Accuracy: 0.9688
Epoch 6 | Loss: 0.1180 | Test Accuracy: 0.9740
Epoch 7 | Loss: 0.1093 | Test Accuracy: 0.9746
Epoch 8 | Loss: 0.1058 | Test Accuracy: 0.9748
Epoch 9 | Loss: 0.0975 | Test Accuracy: 0.9738
Epoch 10 | Loss: 0.0891 | Test Accuracy: 0.9732


In [18]:
# 최종 정확도 출력
final_acc = evaluate(model, test_loader)
print(f"\n최종 정확도: {final_acc*100:.2f}%")


최종 정확도: 97.32%
